In [132]:
using PlotlyJS
# using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
# XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))

calculate_reserve_gcd_KPI (generic function with 1 method)

In [133]:
folder_path = joinpath("..","output", "solutions_v14.3u")
aux = "all"
# aux = join(string.("n_",[7, 228, 259, 289]),"_")

gcdi_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_adequacy.parquet"))
gcd_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_adequacy.parquet"))
KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_KPI_reserve.parquet"))
gcdi_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_reserve.parquet"))
gcd_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_reserve.parquet"))


;

../output/solutions_v14.3u/all_gcdi_KPI_adequacy.parquet
../output/solutions_v14.3u/all_gcd_KPI_adequacy.parquet
../output/solutions_v14.3u/all_KPI_reserve.parquet
../output/solutions_v14.3u/all_gcdi_KPI_reserve.parquet
../output/solutions_v14.3u/all_gcd_KPI_reserve.parquet


In [134]:
reserve_thrs = 0.1
group_by = [:configuration, :day, :iteration]

ov = leftjoin(
    gcdi_KPI_adequacy[:,union(group_by,[:objective_value, :OPEX, :ENS_MWh])],
    combine(groupby(KPI_reserve, group_by), :required_r_relative => (x-> all(abs.(x) .<= reserve_thrs))  => :required_reserve_within_margins),
    on = group_by
)

eov = combine(groupby(ov, [:configuration, :day, :required_reserve_within_margins]), [:objective_value, :OPEX, :ENS_MWh] .=> mean .=> [:EOV, :EOPEX, :EENS])
sort!(transform!(eov, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu), :mu)
;

In [135]:
plot(eov, x = :mu, y = :EOV, color = :day, facet_col = :required_reserve_within_margins)
# p2 = plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)


data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [136]:
plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [137]:
function redispatch_prob(r_relative_min, r_relative_max, required_reserve, number_key)
    out =  combine(groupby(required_reserve, [:configuration, :mu]), [:required_r_relative, :redispatch_needed] => ((x,y) -> 100*sum((x.>=r_relative_min).*(x.<=r_relative_max).*y)/sum((x.>=r_relative_min).*(x.<=r_relative_max))) => :redispatch_probability)
    out.r_max_relative .= number_key
    return out
end

redispatch_prob (generic function with 1 method)

KPI_reserve

In [138]:
KPI_reserve

Row,configuration,day,iteration,hour,demand_MW,LOL_MW,LGEN_MW,demand_uc_MW,required_r_MW,required_r_relative,redispatch_MW,redispatch_relative,redispatch_needed,required_r_up_MW,required_r_dn_MW,production_MW,charge_MW,discharge_MW,delivered_r_up_MW,delivered_r_dn_MW,delivered_r_up_ratio,delivered_r_dn_ratio,mu
,Symbol,Int64,Symbol,Int64?,Float64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Bool?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64?
1,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,25,1560.06,0.0,0.0,1451,109.062,0.0751635,0.0,0.0,false,109.062,-0.0,1482.78,0.0,77.2855,109.062,-0.0,1.0,NaN,0.0
2,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,26,1408.78,121.676,0.0,1377,153.452,0.11144,121.676,0.0883628,true,153.452,-0.0,1408.78,0.0,0.0,31.7767,-0.0,0.207079,NaN,0.0
3,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,27,1312.67,0.0,97.3265,1410,-97.3265,-0.0690259,-97.3265,-0.0690259,true,-0.0,97.3265,1410.0,0.0,0.0,2.27374e-13,-0.0,-Inf,-0.0,0.0
4,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,28,1481.71,0.0,6.2856,1488,-6.2856,-0.00422419,-6.2856,-0.00422419,true,-0.0,6.2856,1488.0,0.0,0.0,2.27374e-13,-0.0,-Inf,-0.0,0.0
5,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,29,1623.61,0.0,0.0,1669,-45.3852,-0.027193,0.0,0.0,false,-0.0,45.3852,1453.65,34.5517,204.513,-0.0,45.3852,NaN,1.0,0.0
6,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,30,1931.53,0.0,0.0,1931,0.529953,0.000274445,0.0,0.0,false,0.529953,-0.0,1498.58,0.0,432.951,0.529953,-0.0,1.0,NaN,0.0
7,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,31,2190.72,0.0,0.0,2182,8.7232,0.0039978,0.0,0.0,false,8.7232,-0.0,1482.32,0.0,708.407,8.7232,-0.0,1.0,NaN,0.0
8,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,32,2508.8,0.0,0.0,2302,206.8,0.0898351,0.0,0.0,false,206.8,-0.0,1808.93,0.0,699.869,206.8,-0.0,1.0,NaN,0.0
9,base_ramp_storage_envelopes_up_0_dn_0,1,demand_86,33,2561.6,0.0,0.0,2421,140.602,0.0580762,0.0,0.0,false,140.602,-0.0,2673.59,113.585,1.60162,140.602,-0.0,1.0,NaN,0.0


In [139]:
combine(groupby(KPI_reserve, [:configuration, :mu]), [:required_r_relative, :redispatch_needed] => ((x,y) -> sum((x.>=0).*(x.<=0.1))) => :redispatch_probability)
# 
# CSV.write("KPI_reserve_v12_0.csv", KPI_reserve)

Row,configuration,mu,redispatch_probability
,Symbol,Float64?,Int64
1,base_ramp_storage_envelopes_up_0_dn_0,0.0,8159
2,base_ramp_storage_envelopes_up_0_2_dn_0_2,0.2,8159
3,base_ramp_storage_envelopes_up_0_4_dn_0_4,0.4,8159
4,base_ramp_storage_envelopes_up_0_6_dn_0_6,0.6,8159
5,base_ramp_storage_envelopes_up_0_8_dn_0_8,0.8,8159
6,base_ramp_storage_envelopes_up_1_dn_1,1.0,8159
7,base_ramp_storage_energy_reserve_cumulated,missing,8159


In [140]:
redispatch_probability = vcat(
    vcat([redispatch_prob(0, r_max, KPI_reserve, r_max) for r_max in 0.0:0.01:0.15]...),
    vcat([redispatch_prob(-r_max, 0, KPI_reserve, -r_max) for r_max in 0.0:0.01:0.15]...)
)
sort!(redispatch_probability, :r_max_relative)
# redispatch_probability = vcat([redispatch_prob(-r_max, r_max, required_reserve, r_max) for r_max in 0.0:0.01:0.2]...)
p1 = plot(redispatch_probability, x = :r_max_relative, y = :redispatch_probability, color = :mu, Layout(title = "Redispatch probability", yaxis_title = "probability  [%]", xaxis_title = "required reserve [%]"), mode="markers+lines")
p2 = plot(filter(:mu => x -> ismissing(x) ? false : x == 1, redispatch_probability), x = :r_max_relative, y = :redispatch_probability, Layout(title = "Redispatch probability mu = 1 ", yaxis_title = "probability  [%]", xaxis_title = "required reserve [%]"), name = "1.0", mode="markers+lines", marker=attr(color="black"))
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields marker, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [141]:
p2 = plot(filter(:day => x -> x in [7], KPI_reserve), x = :required_r_relative, y = :redispatch_MW, color = :mu, mode = "markers", Layout(height=800))

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, legend, margin, template, xaxis, and yaxis"

In [142]:
p2 = plot(filter(:day => x -> x in [7], KPI_reserve), x = :required_r_relative, y = :mu, facet_col = :redispatch_needed, facet_row = :day, mode = "markers", Layout(height=800))
# p2 = plot(filter(:mu => x -> ismissing(x) ? false : x == 1, KPI_reserve), x = :required_r_relative, y = :mu, facet_col = :redispatch_needed, mode = "markers", Layout(height=800))

data: [
  "scatter with fields mode, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [143]:
# KPI_reserve[:KPI_reserve.mu .== 1]

In [144]:
p1 = plot(gcd_KPI_adequacy, x = :mu, y = :EOV, color = :day)
p2 = plot(gcd_KPI_adequacy, x = :mu, y = :EENS, color = :day)
p3 = plot(gcd_KPI_adequacy, x = :mu, y = :ECUR, color = :day)
p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p1 p2 p4]
# savefig([p1 p2 p3 p4], "adecuacy KPIs v1.png", width=12*scale, height=6*scale)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, xaxis3, yaxis1, yaxis2, and yaxis3"

In [145]:
p1 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_up_ratio, color = :day)
p2 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_dn_ratio, color = :day)
p3 = plot(gcd_KPI_reserve, x = :mu, y = :E_delivered_r_up_MWh, color = :day)
p4 = plot(gcd_KPI_reserve, x = :mu, y = :E_deliverered_dn_MWh, color = :day)
# [p1 p2; p3 p4]
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [146]:
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, color = :iteration)
# p2 = plot(gcdi_KPI_adequacy, y = :ENS_MWh, x = :mu, color = :iteration)
# p3 = plot(gcdi_KPI_adequacy, y = :LGEN_MWh, x = :mu, color = :iteration)
# p4 = plot(gcdi_KPI_adequacy, y = :CUR_MWh, x = :mu, color = :iteration)
# [p1 p2; p3 p4]

In [147]:
p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, facet_col = :day, color = :iteration)
[p1;]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [148]:
# function check_loq_value(df, field, value, accuracy)
#     filter = (df[!,field] .> value) .& (df[!,field] .< Inf) .& (.!isnan.(df[!,field]))
#     filter = (df[!,field].*filter .-value) .< accuracy
#     println(all(filter))
#     return df[.!filter,:]

# end
# # Check
# accuracy = 0.000001
# check_loq_value(KPI_reserve, :delivered_r_up_ratio, 1, accuracy)
# check_loq_value(KPI_reserve, :delivered_r_dn_ratio, 1, accuracy)
;
# balance = (KPI_reserve.required_r_up_MW .-KPI_reserve.required_r_dn_MW) .- (KPI_reserve.delivered_r_up_MW .- KPI_reserve.delivered_r_dn_MW .+ KPI_reserve.LOL_MW.*(required.>=0))
# println(all(abs.(balance) .<= accuracy))
# Todo: re-calcualte balance balance
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration) 
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_valuef, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)
# p1 = plot(gcdi_KPI_adequacy, y = :objective_value, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p2 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p3= plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# relayout([p1;p2;p3], height=800)

### Checking reserve delivey

In [149]:
day_ = 7
iteration_ = :demand_40
config_ = :base_ramp_storage_envelopes_up_0_3_dn_0_3
KPI_reserve_ = KPI_reserve[(KPI_reserve.configuration .== config_) .& (KPI_reserve.day .== day_) .& (KPI_reserve.iteration .== iteration_), :]
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_up_MW, name = "required_r_up_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_up_MW, name = "delivered_r_up_MW")]
p1 = plot(p, Layout(yaxis_title="reserve up MW", xaxis_title="hour"))
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_dn_MW, name = "required_r_dn_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_dn_MW, name = "delivered_r_dn_MW")]
p2 = plot(p, Layout(yaxis_title="reserve dn MW", xaxis_title="hour"))
[p1 p2]

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [150]:
group_by_big= [:configuration, :day]
reserve_ = combine(groupby(s_uc.reserve, union(group_by_big, [:resource])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_MW,:reserve_down_MW])
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW])
leftjoin!(reserve_,
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW]),
on = group_by_big
)

reserve_.reserve_down_relative = reserve_.reserve_down_MW ./ reserve_.reserve_down_total_MW
reserve_.reserve_up_relative = reserve_.reserve_up_MW ./ reserve_.reserve_up_total_MW
transform!(reserve_, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
sort!(reserve_, :mu)
;

LoadError: UndefVarError: `s_uc` not defined

In [ ]:
fieldy = :resource
fieldx = :reserve_up_relative
df = reserve_[reserve_.day .== 7,:] 
plot([scatter(
          x= df[df[!,fieldy] .== r, :mu], y=df[df[!,fieldy] .== r, fieldx],
          stackgroup=map_stack_group(r,"required"), mode="lines", name = r,
          line=attr(width=1, color=color_discrete_map(r), shape = "line")
      ) for r in unique(df[!,fieldy])],
    Layout(yaxis_title=string(fieldx), xaxis_title=string(:mu), title = string(fieldx))
)
